In [14]:
import pandas as pd, tensorflow as tf, matplotlib.pyplot as plt, numpy as np, os
from   sklearn.model_selection import train_test_split
from   sklearn.metrics import mean_squared_error
from   tensorflow import keras
from   math import sqrt
from keras.models import load_model
from tensorflow.keras.callbacks import ModelCheckpoint

In [9]:
#Data Base Import
db=pd.read_excel('db.xlsx')

#Data Extraction
y1 = db ["As_opt"]
y2 = db ["d_opt"]
y3 = db ["As_opt_p"]
x  = db [["fc","fy","Mu","b"]]

#RMSE def
def rmse(ytrue,ypred):
  return tf.sqrt(tf.reduce_mean(tf.square(ytrue - ypred)))

In [10]:
#Split: DataBase into: Training and Testing Data
xtrain,xtest,ytrain1,ytest1,ytrain2,ytest2,ytrain3,ytest3 = train_test_split(x,y1,y2,y3,test_size=0.2,random_state=1) 

In [19]:
# Auto-Saving Model Training: Callback ModelCheckpoint
checkpoint = ModelCheckpoint(r'D:\ToolBox\py\proyect_1\RedesNeuronales\mdl_checkpoint.keras', monitor='val_loss', save_best_only=True)

In [24]:
epochs = 1
# Define el modelo ANN con tres salidas
mdl = tf.keras.Sequential([
    tf.keras.layers.Dense(40, activation="relu", input_shape=(xtrain.shape[1],)),
    tf.keras.layers.Dense(40, activation="relu"),
    tf.keras.layers.Dense(40, activation="relu"),
    tf.keras.layers.Dense(40, activation="relu"),
    # Capas de salida para cada una de las tres salidas
    tf.keras.layers.Dense(1, name="As_opt"),  # Primera salida
    tf.keras.layers.Dense(1, name="d_opt"),  # Segunda salida
    tf.keras.layers.Dense(1, name="As_opt_p")   # Tercera salida
])

c:\Users\JORGE\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [ ]:
# Stats: Model 
mdl.summary()
print(mdl.weights)

In [25]:
# Model Conpiling
mdl.compile(optimizer='adam', loss=rmse ,metrics=['accuracy'])

In [28]:
# Model Parameters Training 
mdl_fit = mdl.fit(xtrain, {"As_opt": ytrain1, "d_opt": ytrain2, "As_opt_p": ytrain3},
    epochs=epochs,
    verbose=True, 
    validation_split=0.05,
    callbacks=[checkpoint])

2494/2494 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step - accuracy: 0.0000e+00 - loss: 0.3865 - val_accuracy: 0.0000e+00 - val_loss: 0.2665


In [29]:
# Stats: Model Parameters Training 
print(mdl_fit.history)

{'accuracy': [0.0], 'loss': [0.3762800395488739], 'val_accuracy': [0.0], 'val_loss': [0.2665410041809082]}


In [ ]:
# COMPARING PREDICTED VALUES VS THEORETICAL VALUES
predictions = mdl.predict(xtrain)

# Las predicciones serán una lista de arrays, uno por cada salida
ypred1 = predictions[0]  # Predicciones para As_opt
ypred2 = predictions[1]  # Predicciones para d_opt
ypred3 = predictions[2]  # Predicciones para As_opt_p

RMSE=sqrt(mean_squared_error(ytrain1,ypred1))
RMSE=sqrt(mean_squared_error(ytrain2,ypred2))
RMSE=sqrt(mean_squared_error(ytrain3,ypred3))

# Coefficients
m1, b1 = np.polyfit(ytrain1, ypred1, 1)
m2, b2 = np.polyfit(ytrain2, ypred2, 1)
m3, b3 = np.polyfit(ytrain3, ypred3, 1)

#Plotting scatter points and 1st-degree polynom
# Information Plot
plt.scatter(ypred1,ytrain1,facecolors = 'none',edgecolors='b')
plt.plot(ytrain1, m1*ytrain1 + b1, color='red')
plt.text(0.45, 0.2, f'$y = {m1} \cdot x + {b1}$', fontsize=10, transform=plt.gca().transAxes)
plt.text(0.45, 0.15, f'RMSE={RMSE}', fontsize=10, transform=plt.gca().transAxes)
plt.xlabel('ANN predicted reinforcement area As (cm^2)')
plt.ylabel('Calculated reinforcement area As (cm^2)')
plt.title('Model 3 layers & 20 neurons // Data Training (Epoch = 150)')

plt.scatter(ypred2,ytrain2,facecolors = 'none',edgecolors='b')
plt.plot(ytrain2, m2*ytrain2 + b2, color='red')
plt.text(0.45, 0.2, f'$y = {m2} \cdot x + {b2}$', fontsize=10, transform=plt.gca().transAxes)
plt.text(0.45, 0.15, f'RMSE={RMSE}', fontsize=10, transform=plt.gca().transAxes)
plt.xlabel('ANN predicted depth d (cm)')
plt.ylabel('Calculated depth d (cm)')
plt.title('Model 3 layers & 20 neurons // Data Training (Epoch = 150)')	

plt.scatter(ypred1,ytrain1,facecolors = 'none',edgecolors='b')
plt.plot(ytrain3, m3*ytrain3 + b3, color='red')
plt.text(0.45, 0.2, f'$y = {m3} \cdot x + {b3}$', fontsize=10, transform=plt.gca().transAxes)
plt.text(0.45, 0.15, f'RMSE={RMSE}', fontsize=10, transform=plt.gca().transAxes)
plt.xlabel('ANN predicted reinforcement area As (cm^2)')
plt.ylabel('Calculated reinforcement area As (cm^2)')
plt.title('Model 3 layers & 20 neurons // Data Training (Epoch = 150)')